# Imports:

# Code: 

### Imports

In [1]:
import cv2
import mediapipe as mp 
import numpy as np 
from enum import IntEnum
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_hand = mp.solutions.hands


objc[17947]: Class CaptureDelegate is implemented in both /Users/danielzhao/miniconda3/envs/mediapipe/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x162112620) and /Users/danielzhao/miniconda3/envs/mediapipe/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x1473d8860). One of the two will be used. Which one is undefined.
objc[17947]: Class CVWindow is implemented in both /Users/danielzhao/miniconda3/envs/mediapipe/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x162112670) and /Users/danielzhao/miniconda3/envs/mediapipe/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1260f4a68). One of the two will be used. Which one is undefined.
objc[17947]: Class CVView is implemented in both /Users/danielzhao/miniconda3/envs/mediapipe/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x162112698) and /Users/danielzhao/miniconda3/envs/mediapipe/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1260f4a90). One 

### Calculations

##### Calculate Angle

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

##### Get Leg Angles

In [3]:
def getAngles(landmarks):
    hip_left = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    knee_left = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    ankle_left = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

    hip_right = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    knee_right = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    ankle_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

    angle_left = calculate_angle(hip_left, knee_left, ankle_left)
    angle_right = calculate_angle(hip_right, knee_right, ankle_right)

    return [angle_left, angle_right]

#### Handle and Draw UI

In [2]:
class GameStates(IntEnum):
    PAUSE = -1
    MAIN_MENU = 0
    READY = 1
    START_ROUND = 2
    END_ROUND = 3
    GAME_OVER = 4
    
#set current ui to main menu
gameState = GameStates.READY

In [6]:
def drawUI(screen):
    if gameState == 0:
        pass

#### Game loop

In [3]:
# setup camera
cam = cv2.VideoCapture(0)

pose = mp_pose.Pose(min_detection_confidence=0.80, min_tracking_confidence=0.80)
hands = mp_hand.Hands(min_detection_confidence=0.80, min_tracking_confidence=0.80)
while cam.isOpened():
    ret, frame = cam.read()
    if cv2.waitKey(10) & 0xFF == ord('p'):
        break

    # proccess image, convert from BRG to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    if gameState == GameStates.MAIN_MENU:
        # proccess image through hand detection
        hand_results = hands.process(image)
    elif gameState == GameStates.READY:
        # crop photo in half horizontally
        image_left = image[ :, :image.shape[1]//2]
        image_left.flags.writeable = False

        image_right = image[ :, image.shape[1]//2:]
        image_right.flags.writeable = False

        hands_results_left = hands.process(np.ascontiguousarray(image_left))
        hands_results_right = hands.process(np.ascontiguousarray(image_right))

        cv2.imshow("Testing1", image_left)
        cv2.imshow("Testing2", image_right)
    
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
    cv2.imshow('QuickDraw', image)
    
cam.release()
cv2.destroyAllWindows()   

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960, 3)
(1080, 960

: 

### Detections

In [ ]:
# squat counter variables
counter = 0

leg = 0 # 0 -> none, 1 -> left, 2 -> right
halfDone = False

# debugging mode/display detections
debugMode = False


#setup mediapipe, 75%
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.75) as pose:
    while cam.isOpened():
        ret, frame = cam.read()
        
        # BRG -> RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        results = pose.process(image)

        # Get landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
    
        # RGB -> BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # get landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            #get angles
            leftAngle, rightAngle = getAngles(landmarks)

            # Determine which leg is being used for
            if leftAngle < 170 and not rightAngle < 160:
                leg = 1
            elif rightAngle < 170 and not rightAngle < 160:
                leg = 2
            
            



            
            # show angle
            leftKnee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            rightKnee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            cv2.putText(image, str(leftLeg), 
                        tuple(np.multiply(leftKnee, [frame.shape[0], frame.shape[1]]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(rightLeg), 
                        tuple(np.multiply(rightKnee, [frame.shape[0], frame.shape[1]]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        cv2.imshow('Mediapipe Feed', image)

        # get input
        if cv2.waitKey(10) & 0xFF == ord('p'):
            break

    cam.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


: 

: 

: 